In [1]:
using Revise
using ITensors,ITensorMPS
using PastaQ
push!(LOAD_PATH,"../src/")
using RandomMeas
using ProgressBars


In [2]:
N = 10
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = apply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = randomMPS(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-12.349130849240838  maxlinkdim=4 maxerr=1.90E-16 time=13.251
After sweep 2 energy=-12.381380862342644  maxlinkdim=13 maxerr=5.40E-11 time=0.010
After sweep 3 energy=-12.381489987194616  maxlinkdim=10 maxerr=9.51E-11 time=0.041
After sweep 4 energy=-12.381489996566899  maxlinkdim=8 maxerr=8.51E-11 time=0.010
After sweep 5 energy=-12.381489996620886  maxlinkdim=8 maxerr=7.00E-11 time=0.010

Ground state energy:  -12.381489996620886

---------------------------------------



In [3]:
η = 10
m = 4
# Build the gate structure
circuit = []
for d in 1:η
    xx_layer = [("Rxx", (j, j + 1), (ϕ=1*d/η/m,)) for j in 1:(N - 1)]
    z_layer = [("Rz", j, (θ=2*B/m,)) for j in 1:N]
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);

In [4]:
ψt = runcircuit(ψ0, circuit; cutoff=1E-8)
orthogonalize!(ψt,1)
ψt[1] /= norm(ψt[1])
E_th = real(inner(ψt', H,ψt))
println("final energy ",E_th)
E2_th = real(inner(ψt', H2,ψt))-E_th^2
println("final energy variance ",E2_th)

final energy -11.869405413372245
final energy variance 2.427253542583628


In [7]:
Nu = 1000
NM = 1000

In [9]:
Es = 0.
E2s = 0.
for r in ProgressBar(1:Nu, printing_delay=5)
        u = get_rotations(ξ,1)
        data = get_RandomMeas_MPS(ψt,u,NM)
        for m in 1:NM
            ρ = get_shadow_factorized(data[m,:],ξ,u)
            Es += get_expect_shadow(H,ρ,ξ)/Nu/NM
            E2s += get_expect_shadow(H2,ρ,ξ)/Nu/NM
        end
end
E2s -= Es^2
println("estimated energy ",Es)
println("estimated energy variance ",E2s)

0.0%┣                                             ┫ 0/1.0k [00:00<00:00, -0s/it]
0.4%┣▏                                             ┫ 4/1.0k [00:05<29:27, 2s/it]
1.7%┣▊                                            ┫ 17/1.0k [00:11<10:47, 2it/s]
3.0%┣█▍                                           ┫ 30/1.0k [00:16<08:55, 2it/s]
4.3%┣██                                           ┫ 43/1.0k [00:21<08:03, 2it/s]
5.5%┣██▌                                          ┫ 55/1.0k [00:26<07:41, 2it/s]
6.5%┣███                                          ┫ 65/1.0k [00:31<07:39, 2it/s]
7.5%┣███▍                                         ┫ 75/1.0k [00:37<07:38, 2it/s]
8.6%┣███▉                                         ┫ 86/1.0k [00:42<07:31, 2it/s]
9.4%┣████▎                                        ┫ 94/1.0k [00:48<07:45, 2it/s]
10.0%┣████▎                                      ┫ 100/1.0k [00:54<08:06, 2it/s]
10.6%┣████▋                                      ┫ 106/1.0k [00:59<08:20, 2it/s]
11.0%┣████▊                 

estimated energy -11.820477436053348
estimated energy variance 3.4909706459155814


100.0%┣█████████████████████████████████████████┫ 1.0k/1.0k [08:28<00:00, 2it/s]
100.0%┣█████████████████████████████████████████┫ 1.0k/1.0k [08:28<00:00, 2it/s]
